In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import numpy as np # linear algebra
import pandas # data processing, CSV file I/O (e.g. pd.read_csv)
from pprint import pprint
import time
import io
import os
import re
from pprint import pprint
g = 0

thisyear = 1929
league = 'N'

just_this_team = 'TOR' #['ANA', 'BAL']
just_this_team_order = 14

just_this_team_flag = False
if (thisyear >= 1960):
    just_this_team_flag = True

pandas.set_option('display.max_colwidth', None)

eventdir = '../input/baseball-events-from-retrosheetorg/events/'

processed_data_dir = '../input/baseball-events-from-retrosheetorg/'

save_fielders = True

print('initializing dataframes...')
event_id_columns = ['eventID']
gamecolumns = ['gameID']
info_columns = gamecolumns + ['visteam', 'hometeam', 'site', 'date', 'number', 'starttime', 'daynight', 'usedh', 'innings', 'tiebreaker', 'umphome', 'ump1b', 'ump2b', 'ump3b', 'umplf', 'umprf', 'inputtime', 'howscored', 'pitches', 'oscorer', 'temp', 'winddir', 'windspeed', 'fieldcond', 'precip', 'sky', 'timeofgame', 'attendance', 'wp', 'lp', 'save', 'comments']
player_columns = ['playerID', 'visitor_or_home', 'batting_order', 'fielding_position']

event_columns = event_id_columns + gamecolumns
event_columns += ['event_in_game', 'inning', 'visitor_or_home', 'batterID', 'batter_hand']
event_columns += ['pitch_count', 'pitch_sequence', 'theplay']
event_columns += ['pitcherID', 'pitcher_hand', 'badj', 'padj', 'ladj']
if (save_fielders):
    event_columns += ['field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_3B_playerID', 'field_SS_playerID', 'field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID', 'bat_DH_playerID']
event_columns += ['pinch_runner_playerID', 'pinch_runner_replaced_playerID']#, 'badj', 'padj']
event_columns += ['comments']

def ordinaltg(n):
    return str(n) + {1: 'st', 2: 'nd', 3: 'rd'}.get(4 if 10 <= n % 100 < 20 else n % 10, "th")

g = 0
print('Packages loaded!')

initializing dataframes...
Packages loaded!


In [18]:
s = time.time()

show_all_games = False
show_all_lines = False

start_at_line = 0

game_info_all_df = pandas.DataFrame(data=None, columns=info_columns).set_index('gameID')
game_data_all_df = pandas.DataFrame(data=None, columns=['gameID'])#.set_index('gameID')
er_all_df = pandas.DataFrame(data=None, columns=['gameID', 'pitcher', 'er'])

events_all_df = pandas.DataFrame(data=None, columns=event_columns).set_index('eventID')

players_all_df = pandas.DataFrame(data=None, columns=player_columns).set_index(['playerID'])

comments_at_events = []
pinch_runners_at_events = []
badj_at_events = []
padj_at_events = []
ladj_at_events = []

all_lines_list = []

for i in range(thisyear, thisyear+1):
    
    game_info_thisyear_df = pandas.DataFrame(data=None, columns=game_info_all_df.columns)
#    game_data_thisyear_df = pandas.DataFrame(data=None, columns=game_data_all_df.columns)
    er_thisyear_df = pandas.DataFrame(data=None, columns=er_all_df.columns)
    players_thisyear_df = pandas.DataFrame(data=None, columns=players_all_df.columns)
    events_thisyear_df = pandas.DataFrame(data=None, columns=events_all_df.columns)
    
    thisyear_files = sorted(['{0:}{1:.0f}seve/'.format(eventdir, np.floor(i/10)*10)+x for x in os.listdir('{0:}{1:.0f}seve/'.format(eventdir, np.floor(i/10)*10)) if ((str(i) in x) & (x[-4:] == '.EV{0:}'.format(league)))])
    print('Found {0:,.0f} event files...'.format(len(thisyear_files)))
    
    if (just_this_team_flag):
        start_at_file_number = just_this_team_order
        end_at_file_number = just_this_team_order + 1
    else:
        start_at_file_number = 0
        end_at_file_number = len(thisyear_files)
        
    for thisfile in thisyear_files[start_at_file_number:end_at_file_number]:
        game_info_df = pandas.DataFrame(data=None, columns=game_info_thisyear_df.columns)
#        game_data_df = pandas.DataFrame(data=None, columns=game_data_thisyear_df.columns)
        er_df = pandas.DataFrame(data=None, columns=er_thisyear_df.columns)
        events_df = pandas.DataFrame(data=None, columns=events_thisyear_df.columns)
        players_df = pandas.DataFrame(data=None, columns=players_thisyear_df.columns)

        all_lines_list = []
        print('Reading {0:}...'.format(thisfile.split('/')[-1]))
        with io.open(thisfile, 'r', encoding='utf-8') as f:
            orig_lines_list = f.readlines()
        for this_orig_line in orig_lines_list:
            all_lines_list += [x for x in this_orig_line.split('\n')]
        all_lines_list = [x for x in all_lines_list 
                          if ((x != '') and (x != '\x1a'))
                         ]

        #print('\tRead {0:,.0f} lines...'.format(len(all_lines_list)))
        
        event_dict = {}
        gamecount = 0
        eventcount = 0
        the_event_in_game = 0
        batting_team = 0
        comment_text = ''
        game_info_comment_text = ''

        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
        staged_play = False
        
        for i in range(start_at_line, len(all_lines_list)): #range(0, len(all_lines_list)):
            if (show_all_lines):
                print('\tParsing line {0:,.0f} of {1:,.0f}...'.format(i, len(all_lines_list)))
                print(i, all_lines_list[i])
            elif (np.mod(i,1000) == 0):
                print('\tParsing line {0:,.0f} of {1:,.0f}...'.format(i, len(all_lines_list)))
            record_type = all_lines_list[i].split(',')[0]
            #print(record_type)
            if (record_type == 'id'):   
                watchflag = False
#                 if (all_lines_list[i].split(',')[1] == 'PHA191807062'):
#                     watchflag = True
                if (gamecount >= 1):   #WRITE LAST PLAY OF PREVIOUS GAME!
                    if (comment_text != ''):
                        event_dict['comments'] = comment_text
                    else:
                        event_dict['comments'] = np.nan
                    event_dict['pinch_runner_playerID'] = pinch_runner
                    event_dict['pinch_runner_replaced_playerID'] = pinch_runner_replaced
                    if (pinch_runner_flag == True):
                        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
                    events_df.loc[eventcount] = event_dict
                    eventcount += 1
                    the_event_in_game == 1
                    comment_text = ''

                    if (game_info_comment_text != ''):
                        this_game_info['comments'] = game_info_comment_text
                        game_info_comment_text = ''
                    else:
                        this_game_info['comments'] = np.nan
                    game_info_df.loc[thegameID] = this_game_info

                gamecount += 1        
                thegameID = all_lines_list[i].split(',')[1]
                if (show_all_games):
                    print('\t\tReading {0:} (starts on line {1:.0f})...'.format(thegameID, i))
                elif (np.mod(gamecount, 50) == 0):
                    print('\t\tReading {0:} (starts on line {1:.0f})...'.format(thegameID, i))

                players_df = players_df[players_df['visitor_or_home'].apply(lambda x: True == False)]

        #         if (thegameID == 'ANA202008100'):
        #             break
                eventcount += 1
                this_game_info = {}
                the_event_in_game = 1
                staged_play = False
#                 if (np.mod(gamecount, 50) == 1):
#                     print('\t\tParsing game {0:.0f}: {1:} (i = {2:,.0f}; eventcount = {3:,.0f})...'.format(gamecount, thegameID, i, eventcount))
        #            print('Starting Top of the 1st...')
            elif (record_type == 'version'):
                this_game_info['version'] = all_lines_list[i].split(',')[1]
            elif (record_type == 'info'):
                this_game_info[all_lines_list[i].split(',')[1]] = all_lines_list[i].split(',')[2]
            elif (record_type == 'start'):
                this_player_df = pandas.DataFrame(data=[[all_lines_list[i].split(',')[1], int(all_lines_list[i].split(',')[3]), int(all_lines_list[i].split(',')[4]), int(all_lines_list[i].split(',')[5])]], columns=player_columns).set_index('playerID')
                players_df = pandas.concat((players_df, this_player_df), axis=0)
            elif (record_type == 'data'):
                try:
                    this_er_row_df = pandas.DataFrame(data=[[thegameID, all_lines_list[i].split(',')[2], int(all_lines_list[i].split(',')[3])]], columns=er_df.columns)
                except ValueError:
                    this_er_row_df = pandas.DataFrame(data=[[thegameID, all_lines_list[i].split(',')[2], int(all_lines_list[i].split(',')[3][0:2])]], columns=er_df.columns) # Catching one line (BAL2021 line 6015) with a stray "1    a"
                er_df = pandas.concat((er_df, this_er_row_df))
            elif (record_type == 'play'):
                if ((watchflag) & (all_lines_list[i].split(',')[-1] == 'NP')):
                    print('{0:0d}: {1:}: {2:}'.format(i, all_lines_list[i], all_lines_list[i].split(',')[-1]))
                if (staged_play):
                    if (comment_text != ''):
                        event_dict['comments'] = comment_text
                    else:
                        event_dict['comments'] = np.nan
                    event_dict['pinch_runner_playerID'] = pinch_runner
                    event_dict['pinch_runner_replaced_playerID'] = pinch_runner_replaced
                    if (pinch_runner_flag == True):
                        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
                    events_df.loc[eventcount] = event_dict
                    eventcount += 1
                    the_event_in_game == 1
                    comment_text = ''
                event_dict['event_in_game'] = the_event_in_game
                event_dict['gameID'] = thegameID
                event_dict['inning'] = int(all_lines_list[i].split(',')[1])
                event_dict['visitor_or_home'] = int(all_lines_list[i].split(',')[2])
                event_dict['batterID'] = all_lines_list[i].split(',')[3]
                event_dict['pitch_count'] = all_lines_list[i].split(',')[4]
                event_dict['pitch_sequence'] = all_lines_list[i].split(',')[5]
                event_dict['theplay'] = all_lines_list[i].split(',')[6]        
                
                #print(players_df)
                
                event_dict['pitcherID'] = players_df[
                    (players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) 
                    & (players_df['fielding_position'] == 1)
                ].index.values[0]
                #print('\t{0:}'.format(event_dict['pitcherID']))
                if (save_fielders):
                    event_dict['field_C_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 2)].index.values[0]
                    event_dict['field_1B_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 3)].index.values[0]
                    event_dict['field_2B_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 4)].index.values[0]
                    event_dict['field_3B_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 5)].index.values[0]
                    event_dict['field_SS_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 6)].index.values[0]
                    event_dict['field_LF_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 7)].index.values[0]
                    event_dict['field_CF_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 8)].index.values[0]
                    event_dict['field_RF_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 9)].index.values[0]
                                        
#                     if (this_game_info['usedh'] == 'true'):  # lol it's a string, not a boolean
#                         event_dict['bat_DH_playerID'] = players_df[(players_df['visitor_or_home'] == event_dict['visitor_or_home']) & (players_df['fielding_position'] == 10)].index.values[0]
                if (batting_team != event_dict['visitor_or_home']):
        #             if (event_dict['visitor_or_home'] == 0):
        #                 inningside = 'Top'
        #             else:
        #                 inningside = 'Bottom'
        #             print('Starting {0:} of the {1:} - {2:} pitching...'.format(inningside, ordinal(event_dict['inning']), event_dict['pitcherID']))
                    batting_team = 1 - batting_team
                staged_play = True
                the_event_in_game += 1
            elif (record_type == 'com'):
                if (the_event_in_game <= 1):
                    game_info_comment_text += all_lines_list[i][4:]
                if (eventcount not in comments_at_events):
                    comments_at_events.append(eventcount)
                if (staged_play):
                    if (comment_text != ''):
                        comment_text += ' '
                    comment_text += all_lines_list[i][4:]#.replace('"', '').replace('$', '').replace('.', '')
            elif (record_type == 'sub'):
                players_df = players_df.reset_index()
                sub_playerID = all_lines_list[i].split(',')[1]
                if (len(all_lines_list[i].split(',')) == 6):
                    sub_team = int(all_lines_list[i].split(',')[3])
                    sub_batting_order = int(all_lines_list[i].split(',')[4])
                    sub_fielding_position = int(all_lines_list[i].split(',')[5])
                else: # Replace commas in substituted-in player's name
                    commafinder = ''
                    for q in range(2, len(all_lines_list[i].split(','))):
                        try:
                            sub_team = int(all_lines_list[i].split(',')[q])
                            sub_batting_order = int(all_lines_list[i].split(',')[q+1])
                            sub_fielding_position = int(all_lines_list[i].split(',')[q+2])
                            commafinder = commafinder.replace('"','')[:-1]
                            print('\t\tReplacing comma in name of {0:} ({1:})...'.format(commafinder, sub_playerID))
#                             print(commafinder)
#                             print(sub_team)
#                             print(sub_batting_order)
#                             print(sub_fielding_position)
                            break
                        except ValueError:
                                commafinder += all_lines_list[i].split(',')[q] + ' '
                if (sub_team == batting_team):
                    if (sub_fielding_position == 11):
                        pass
                        #event_dict['batterID'] = sub_playerID
                    elif (sub_fielding_position == 12):
                        pinch_runners_at_events.append(eventcount)
                        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [True, sub_playerID, players_df[(players_df['visitor_or_home'] == batting_team) & (players_df['batting_order'] == sub_batting_order)]['playerID'].values[0]]
                    else:
                        players_df.loc[(players_df['visitor_or_home'] == batting_team) & (players_df['batting_order'] == sub_batting_order), 'playerID'] = sub_playerID
                else:
                    players_df.loc[
                        (players_df['visitor_or_home'] == 1-batting_team) 
                        & (players_df['fielding_position'] == sub_fielding_position), 'playerID'] = sub_playerID
                players_df = players_df.set_index('playerID')
            elif (record_type == 'radj'):
                pinch_runners_at_events.append(eventcount)
        #         print('-------------------- RADJ ---------------------------')
        #         print(all_lines_list[i])
                [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [True, all_lines_list[i].split(',')[1], 'radj']
        #         print(pinch_runner_flag, pinch_runner, pinch_runner_replaced)
        #         print('-------------------- /RADJ ---------------------------')
            elif (record_type == 'badj'):

                if (the_event_in_game == 1):
                    #print('\tbadj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                    badj_at_events.append(eventcount)
                else:
                    #print('\tbadj at eventcount {0:,.0f}: {1:}'.format(eventcount+1, all_lines_list[i]))
                    badj_at_events.append(eventcount+1)
            elif (record_type == 'padj'):
                if (the_event_in_game == 1):
                    #print('\tpadj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                    padj_at_events.append(eventcount)
                else:
                    #print('\tpadj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                    padj_at_events.append(eventcount+1)
            elif (record_type == 'ladj'):
                #print('\tladj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                #print(all_lines_list[i])
                ladj_at_events.append(eventcount)
            else:
                print('--------------')
                print(all_lines_list[i-1])     
                print(all_lines_list[i])     
                print(all_lines_list[i+1])     
                print('--------------')
                print('--------------')

        # record last play of final game
        if (comment_text != ''):
            event_dict['comments'] = comment_text
        else:
            event_dict['comments'] = np.nan
        event_dict['pinch_runner_playerID'] = pinch_runner
        event_dict['pinch_runner_replaced_playerID'] = pinch_runner_replaced
        if (pinch_runner_flag == True):
            [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
        events_df.loc[eventcount] = event_dict
        eventcount += 1
        the_event_in_game == 1
        comment_text = ''        

        # record final game info and data
        if (game_info_comment_text != ''):
            this_game_info['comments'] = game_info_comment_text
        else:
            this_game_info['comments'] = np.nan
        game_info_df.loc[thegameID] = this_game_info
        
        events_df.loc[badj_at_events, 'badj'] = True        
        events_df.loc[padj_at_events, 'padj'] = True
        
        game_info_thisyear_df = pandas.concat((game_info_thisyear_df, game_info_df), axis=0)
#        game_data_thisyear_df = pandas.concat((game_data_thisyear_df, game_data_df), axis=0)
        er_thisyear_df = pandas.concat((er_thisyear_df, er_df), axis=0)
        players_thisyear_df = pandas.concat((players_thisyear_df, players_df), axis=0)
        
        events_thisyear_df = pandas.concat((events_thisyear_df, events_df), axis=0)

        print('\n')
        
    game_info_all_df = pandas.concat((game_info_all_df, game_info_thisyear_df), axis=0)
#    game_data_all_df = pandas.concat((game_data_all_df, game_data_thisyear_df), axis=0)
    er_all_df = pandas.concat((er_all_df, er_thisyear_df), axis=0)
    players_all_df = pandas.concat((players_all_df, players_thisyear_df), axis=0)

    events_all_df = pandas.concat((events_all_df, events_thisyear_df), axis=0)

    
#print('Killing in-progress events and replacing with the full file...')

game_info_df = game_info_df[game_info_df['number'].apply(lambda x: True == False)]
er_df = er_df[er_df['er'].apply(lambda x: True == False)]
players_df = players_df[players_df['visitor_or_home'].apply(lambda x: True == False)]
events_df = events_df[events_df['event_in_game'].apply(lambda x: True == False)]

game_info_df = game_info_all_df
er_df = er_all_df
players_df = players_all_df
events_df = events_all_df

er_df = er_df.reset_index(drop=True)
events_df = events_df.reset_index(drop=True)

game_info_df.index.name = 'gameID'
er_df.index.name = 'erID'
players_df.index.name = 'playerID'
events_df.index.name = 'eventID'

# print('backing up...')
game_info_df_bk = game_info_df
er_df_bk = er_df
players_df_bk = players_df
events_df_bk = events_df

e = time.time()
g = g + (e-s)
#print('\n')
print('Parsed {0:,.0f} events in {1:,.0f} minutes {2:,.0f} seconds!'.format(len(events_df), np.floor((e-s)/60), (e-s)%60))


Found 8 event files...
Reading 1929BRO.EVN...
	Parsing line 0 of 11,625...
	Parsing line 1,000 of 11,625...
	Parsing line 2,000 of 11,625...
	Parsing line 3,000 of 11,625...
	Parsing line 4,000 of 11,625...
	Parsing line 5,000 of 11,625...
	Parsing line 6,000 of 11,625...
	Parsing line 7,000 of 11,625...
		Reading BRO192907201 (starts on line 7597)...
	Parsing line 8,000 of 11,625...
	Parsing line 9,000 of 11,625...
	Parsing line 10,000 of 11,625...
	Parsing line 11,000 of 11,625...


Reading 1929BSN.EVN...
	Parsing line 0 of 11,279...
	Parsing line 1,000 of 11,279...
	Parsing line 2,000 of 11,279...
	Parsing line 3,000 of 11,279...
	Parsing line 4,000 of 11,279...
	Parsing line 5,000 of 11,279...
	Parsing line 6,000 of 11,279...
	Parsing line 7,000 of 11,279...
		Reading BSN192907180 (starts on line 7560)...
	Parsing line 8,000 of 11,279...
	Parsing line 9,000 of 11,279...
	Parsing line 10,000 of 11,279...
	Parsing line 11,000 of 11,279...


Reading 1929CHN.EVN...
	Parsing line 0 of 1

In [21]:
def add_roster_entry_if_needed(rosters_df, theplayerID, thenameLast, thenameFirst, thebatting_hand, thethrowing_hand, theteam, theposition, thetheyear):
    if not(theplayerID in rosters_df['playerID'].tolist()):
        rosters_df.loc[rosters_df.index.max()+1] = [theplayerID, thenameLast, thenameFirst, thebatting_hand, thethrowing_hand, theteam, theposition, thetheyear]
        #print('{0:} ({1:} {2:}) already in rosters file...'.format(theplayerID, thenameFirst, thenameLast))
        #pprint(rosters_df[rosters_df['playerID'] == theplayerID])
        #print('----------------------------')
        
    return rosters_df



def add_special_case_roster_entries(rosters_df):
### 1915
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'connt101', 'Connolly', 'Thomas', 'L', 'R', 'WS1', 'RF', 1915)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'conwo101', 'Conway', 'Owen', '?', 'R', 'PHA', '3B', 1915)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'fluhj101', 'Fluhrer', 'John', 'R', 'R', 'CHN', 'LF', 1915)
## FL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'dougl101', 'Douglas', 'Lawrence', 'R', 'R', 'BLF', 'P', 1915)
### 1916
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'carpp101', 'Carpenter', 'Paul', 'R', 'R', 'PIT', 'P', 1916)
### 1918
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'fahef101', 'Fahey', 'Francis', 'B', 'R', 'PHA', 'LF', 1918)
### 1919
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'fishs102', 'Fishburn', 'Samuel', 'R', 'R', 'SLN', 'C', 1919)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'gerne101', 'Gerner', 'Edwin', 'L', 'L', 'CIN', 'P', 1919)
### 1920
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'conwj101', 'Conway', 'Jerome', 'L', 'L', 'WS1', 'P', 1920)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'ellig101', 'Ellison', 'George', 'R', 'R', 'CLE', 'C', 1920)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'davec101', 'Davenport', 'Claude', 'R', 'R', 'NY1', 'P', 1920)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'griet102', 'Griesenbeck', 'Carlos', 'R', 'R', 'SLN', 'C', 1920)
### 1921
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'freeh102', 'Freeman', 'Harvey', 'R', 'R', 'PHA', 'P', 1921)
### 1922
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cox-e101', 'Cox', 'Ernest', 'L', 'R', 'CHA', 'P', 1922)
### 1922
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'eubau101', 'Eubanks', 'Uel', 'R', 'R', 'CHN', 'P', 1922)
    return rosters_df
### 1924
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'gibsc101', 'Gibson', 'Charles', 'R', 'R', 'PHA', 'C', 1924)
### 1927
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'browj102', 'Brown', 'Joseph', 'R', 'R', 'CHA', 'P', 1927)
### 1928
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'grifm102', 'Griffin', 'Martin', 'R', 'R', 'BOS', 'P', 1928)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'greej103', 'Greene', 'Julius', 'L', 'R', 'PHI', 'P', 1928)
### 1929
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'carre101', 'Carroll', 'Edgar', 'R', 'R', 'BOS', 'P', 1929)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cronj103', 'Cronin', 'James', 'B', 'R', 'PHA', '2B', 1929)

    return rosters_df


def fix_special_case_hands(events_df):
# 1915
## AL 
    events_df.loc[events_df['batterID'] == 'colek101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colle101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'connt101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'conwo101', 'batter_hand'] = '?'
    events_df.loc[events_df['batterID'] == 'coumf101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coveh101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'gaind101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'colek101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coumf101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'coveh101', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'chalg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coopw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crutd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'daleg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'darir101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'daubj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'doyll101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fluhj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chalg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coopw101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'crutd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'daleg101', 'pitcher_hand'] = 'R'
## FL
    events_df.loc[events_df['batterID'] == 'camnh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dougl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'downt101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'drakd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'enzej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'falkc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'farrj104', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'kaisg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'camnh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dougl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'falkc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'kaisg101', 'pitcher_hand'] = 'R'
###1916
## AL
    events_df.loc[events_df['batterID'] == 'craws101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dealc102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dumog101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'felsh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gregv101', 'batter_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'connj107', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'corhr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dellw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fleta101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grift102', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dellw101', 'pitcher_hand'] = 'R'
### 1917
## AL
    events_df.loc[events_df['batterID'] == 'culln101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cunng101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davih101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'debeh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gardl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gharp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hamie101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'culln101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cunng101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hamie101', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'carlh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cravg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dillp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'doakb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'everj102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fabrb101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'hemie101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'carlh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'doakb101', 'pitcher_hand'] = 'R'
### 1918
## AL
    events_df.loc[events_df['batterID'] == 'casej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'enzmj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'faber101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'finnh102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hendt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'enzmj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'faber101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'finnh102', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'brott101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'catoh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crumc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dougp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'elleh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fitzm102', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'crumc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dougp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'elleh101', 'pitcher_hand'] = 'R'
### 1919
## AL
    events_df.loc[events_df['batterID'] == 'davii102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dugaj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'evanj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gallb103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gandc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hoytw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kanef101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'gallb103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hoytw101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'cantm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chash101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cruiw101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fishs102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'flacm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gerne101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cantm101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'gerne101', 'pitcher_hand'] = 'L'
### 1920
## AL
    events_df.loc[events_df['batterID'] == 'conwj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'debej101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fortg101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'grifi101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hodgs101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hofmf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'conwj101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'debej101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ellig101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fortg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hodgs101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'davec101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fordh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'griet102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kildp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'davec101', 'pitcher_hand'] = 'R'
### 1921
## AL
    events_df.loc[events_df['batterID'] == 'cobbt101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'erice101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'freeh102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'granj104', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'harph101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'erice101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'freeh102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'harph101', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'connj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'donop102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'filld101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gilhg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'janvh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'keenj103', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'donop102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'filld101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'keenj103', 'pitcher_hand'] = 'L'
### 1922
## AL
    events_df.loc[events_df['batterID'] == 'dursc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ehmkh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fewsc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gallc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jones104', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cox-e101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ehmkh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'jones104', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'cunnb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'duncp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eubau101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grimb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kingl103', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'eubau101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'grimb101', 'pitcher_hand'] = 'R'
### 1923
## AL
    events_df.loc[events_df['batterID'] == 'collr104', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'conns101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'edmog101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hastb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'collr104', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'conns101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'edmog101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hastb101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'carem101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'collp102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cousd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dickl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'doutt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gasta101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'glazw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'collp102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dickl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'glazw101', 'pitcher_hand'] = 'R'
### 1924
## AL
    events_df.loc[events_df['batterID'] == 'daush101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'david102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'drakl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'flagi101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gehrl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gibsc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hargp101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'daush101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'david102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'drakl101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'churj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cloue101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'crith101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ens-j101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fourj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'frisf101', 'batter_hand'] = 'B'
### 1925
## AL
    events_df.loc[events_df['batterID'] == 'englc102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ezzeh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ferga101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ferga101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gregv101', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'bushg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'decaa101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'farrd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bushg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'decaa101', 'pitcher_hand'] = 'R'
### 1926
## AL
    events_df.loc[events_df['batterID'] == 'combe101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'croub101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'falkc102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'foreh101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'foxxj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hadlb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jamic101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'falkc102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'foreh101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'hadlb101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'comoa101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coonj103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cummj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dyere101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'genej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dyere101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'genej101', 'pitcher_hand'] = 'R'
### 1927
## AL
    events_df.loc[events_df['batterID'] == 'cochm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coves101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dixol101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eichi101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fothb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'browj102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coves101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'clarb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cuylk101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deitb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dresc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clarb102', 'pitcher_hand'] = 'R'
### 1928
## AL
    events_df.loc[events_df['batterID'] == 'burkb102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'burng102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clanb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'crowg102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dykej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grifm102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burkb102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'crowg102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'grifm102', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'burwb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cohea101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ehrhr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gonzm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'greej103', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hafec101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burwb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ehrhr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'greej103', 'pitcher_hand'] = 'R'
### 1929
## AL
    events_df.loc[events_df['batterID'] == 'carre101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cissb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cronj103', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'grahs101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grang101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carre101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'grahs101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'grang101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'clarw101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'collp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cvenm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dawsj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'franf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clarw101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cvenm101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dawsj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'franf101', 'pitcher_hand'] = 'R'

    return events_df

print('ok')


ok


In [22]:
s = time.time()
events_df = events_df[events_df['event_in_game'].apply(lambda x: True == False)]
events_df = events_df_bk

print('getting roster file...')
rosters_df = pandas.read_csv(processed_data_dir+'rosters.csv', index_col='rosterID')

print('getting biodata...')
biodata_df = pandas.read_csv(processed_data_dir+'biodata.csv', encoding='utf-8', index_col='bioID')

#print('keeping only this year...')
#rosters_df = rosters_df[(rosters_df['theyear'] == thisyear)]

print('\tadding players not in roster file...')
rosters_df = add_special_case_roster_entries(rosters_df)

# print('backing up rosters...')
# rosters_df_bk = rosters_df


# print('adjusting events file...')
if ('theyear' not in events_df.columns):
    print('\tadding new columns for year and teams...')
    events_df = events_df.assign(theyear = pandas.to_numeric(events_df['gameID'].apply(lambda x: x[3:7]), errors='coerce', downcast='integer'))
if ('visteam' not in events_df.columns):
    events_df = events_df.assign(visteam = events_df.reset_index().merge(game_info_df.reset_index()[['gameID', 'visteam']], how='left', on='gameID').set_index('eventID')['visteam'])
if ('hometeam' not in events_df.columns):
    events_df = events_df.assign(hometeam = events_df.reset_index().merge(game_info_df.reset_index()[['gameID', 'hometeam']], how='left', on='gameID').set_index('eventID')['hometeam'])

print('\tassigning hands...')
events_df.loc[events_df['visitor_or_home'] == 0, 'batter_hand'] = events_df[events_df['visitor_or_home'] == 0].reset_index().merge(rosters_df.reset_index(), how='left', left_on=['batterID', 'theyear', 'visteam'], right_on=['playerID', 'theyear', 'theteam']).set_index('eventID')['bats']
events_df.loc[events_df['visitor_or_home'] == 1, 'batter_hand'] = events_df[events_df['visitor_or_home'] == 1].reset_index().merge(rosters_df.reset_index(), how='left', left_on=['batterID', 'theyear', 'hometeam'], right_on=['playerID', 'theyear', 'theteam']).set_index('eventID')['bats']

print('\tassigning pitcher hands...')
events_df.loc[events_df['visitor_or_home'] == 0, 'pitcher_hand'] = events_df[events_df['visitor_or_home'] == 0].reset_index().merge(rosters_df.reset_index(), how='left', left_on=['pitcherID', 'theyear', 'hometeam'], right_on=['playerID', 'theyear', 'theteam']).set_index('eventID')['throws']
events_df.loc[events_df['visitor_or_home'] == 1, 'pitcher_hand'] = events_df[events_df['visitor_or_home'] == 1].reset_index().merge(rosters_df.reset_index(), how='left', left_on=['pitcherID', 'theyear', 'visteam'], right_on=['playerID', 'theyear', 'theteam']).set_index('eventID')['throws']

print('\tfixing special case hands...')
events_df = fix_special_case_hands(events_df)

print('\taccounting for switch hitters...')
events_df.loc[(events_df['batter_hand'] == 'B') & (events_df['pitcher_hand'] == 'R'), 'batter_hand'] = 'L'
events_df.loc[(events_df['batter_hand'] == 'B') & (events_df['pitcher_hand'] == 'L'), 'batter_hand'] = 'R'

print('\tapplying adjustments to batter and pitcher hands as needed...')
#print('\n')
pitcher_hand_switcher_df = pandas.DataFrame(data=events_df[(events_df['padj'] == True)]['pitcher_hand'], columns=['pitcher_hand'])
pitcher_hand_switcher_df = pitcher_hand_switcher_df.assign(pitcher_hand_switched = np.nan)
pitcher_hand_switcher_df.loc[pitcher_hand_switcher_df['pitcher_hand'] == 'L', 'pitcher_hand_switched'] = 'R'
pitcher_hand_switcher_df.loc[pitcher_hand_switcher_df['pitcher_hand'] == 'R', 'pitcher_hand_switched'] = 'L'
events_df.loc[(events_df['padj'] == True), 'pitcher_hand'] = pitcher_hand_switcher_df['pitcher_hand_switched']

batter_hand_switcher_df = pandas.DataFrame(data=events_df[(events_df['badj'] == True)]['batter_hand'], columns=['batter_hand'])
batter_hand_switcher_df = batter_hand_switcher_df.assign(batter_hand_switched = np.nan)
batter_hand_switcher_df.loc[batter_hand_switcher_df['batter_hand'] == 'L', 'batter_hand_switched'] = 'R'
batter_hand_switcher_df.loc[batter_hand_switcher_df['batter_hand'] == 'R', 'batter_hand_switched'] = 'L'
events_df.loc[(events_df['badj'] == True), 'batter_hand'] = batter_hand_switcher_df['batter_hand_switched']

missing_hand_batters_list = events_df[events_df['batter_hand'].isnull()]['batterID'].drop_duplicates().sort_values().tolist()
missing_hand_batters_list = sorted(missing_hand_batters_list)

print('\n')
print('No batter hand found for {0:,.0f} batters...'.format(len(missing_hand_batters_list)))

missing_hand_pitchers_list = events_df[events_df['pitcher_hand'].isnull()]['pitcherID'].drop_duplicates().sort_values().tolist()
missing_hand_pitchers_list = sorted(missing_hand_pitchers_list)

print('No pitcher hand found for {0:,.0f} pitchers...'.format(len(missing_hand_pitchers_list)))
print('\n')
# for ix, thisrow in rosters_df[
#     (rosters_df['playerID'].isin(missing_hand_batters_list)) 
#     ][['playerID', 'bats']].drop_duplicates().sort_values(by='playerID').iterrows():
#     print("    events_df.loc[events_df['batterID'] == '{0:}', 'batter_hand'] = '{1:}'".format(thisrow['playerID'], thisrow['batting_hand']))

#rosters_df[(rosters_df['playerID'].isin(missing_hand_batters_list))]

print('Searching for batters and pitchers missing from roster files and/or with missing hands...')
add_to_rosters_list = []

b_events_fixer_text = ''
for this_batter in missing_hand_batters_list:
    if (this_batter not in rosters_df['playerID'].drop_duplicates().tolist()):
        #print('{0:} not in roster file'.format(this_batter))
        add_to_rosters_list.append(this_batter)
    batslist = rosters_df[rosters_df['playerID'] == this_batter]['bats'].drop_duplicates().tolist()
    if (len(batslist) == 1):
        bathand = batslist[0]
    elif (len(batslist) == 0):
        if (this_batter in biodata_df.index.tolist()):
            if (biodata_df.loc[this_batter]['bats'] in ['L', 'R', 'B', '?']):
                bathand = biodata_df.loc[this_batter]['bats']
            else:
                bathand = '?'
        else:
            bathand = '?'
    else:
        print('{0:} batted with {1:.0f} different hands: {2:}'.format(this_batter, len(batslist), batslist))
    if (bathand == '?'):
        print("\nNEED TO RESEARCH HAND FOR {0:} {1:} ({2:}) ({3:})".format(biodata_df.loc[this_batter]['nameFirst'], biodata_df.loc[this_batter]['nameLast'], this_batter, bathand))
    b_events_fixer_text += "events_df.loc[events_df['batterID'] == '{0:}', 'batter_hand'] = '{1:}'\n".format(this_batter, bathand)

p_events_fixer_text = ''
for this_pitcher in missing_hand_pitchers_list:
    if (this_pitcher not in rosters_df['playerID'].drop_duplicates().tolist()):
        add_to_rosters_list.append(this_pitcher)
    throwslist = rosters_df[rosters_df['playerID'] == this_pitcher]['throws'].drop_duplicates().tolist()
    if (len(throwslist) == 1):
        throwhand = throwslist[0]
    elif (len(throwslist) == 0):
        if (this_pitcher in biodata_df.index.tolist()):
            if (biodata_df.loc[this_pitcher]['throws'] in ['L', 'R', '?']):
                throwhand = biodata_df.loc[this_pitcher]['throws']
            else:
                throwhand = '?'
        else:
            throwhand = '?'
    else:
        print('{0:} pitched with {1:.0f} different hands: {2:}'.format(this_pitcher, len(throwslist), throwslist))
    if (bathand == '?'):
        print("\nNEED TO RESEARCH HAND FOR {0:} {1:} ({2:}) ({3:})".format(biodata_df.loc[this_pitcher]['nameFirst'], biodata_df.loc[this_pitcher]['nameLast'], this_pitcher, throwhand))
    p_events_fixer_text += "events_df.loc[events_df['pitcherID'] == '{0:}', 'pitcher_hand'] = '{1:}'\n".format(this_pitcher, throwhand)    

print('\n')   

positions_dict = {'field_C_playerID': 'C', 'field_1B_playerID': '1B', 'field_2B_playerID': '2B', 'field_3B_playerID': '3B', 'field_SS_playerID': 'SS', 'field_LF_playerID': 'LF', 'field_CF_playerID': 'CF', 'field_RF_playerID': 'RF', 'pitcherID': 'P', 'bat_DH_playerID': 'DH'}
# position_finder_df = pandas.DataFrame(data=None, columns=positions_dict.keys(), index=add_to_rosters_list)
# position_finder_df.loc[:, :] = 0
preferred_position = ''
roster_fixer_text = ''

for thisone in add_to_rosters_list:
    lastname = biodata_df.loc[thisone]['nameLast']
    firstname = biodata_df.loc[thisone]['nameFirst']
    if (biodata_df.loc[thisone]['bats'] in ['L', 'R', 'B', '?']):
        bathand = biodata_df.loc[thisone]['bats']
    else:
        bathand = '?'
    if (biodata_df.loc[thisone]['throws'] in ['L', 'R', '?']):
        throwhand = biodata_df.loc[thisone]['throws']
    else:
        throwhand = '?'
    teamlist = []
    teamlist += events_df[(events_df['batterID'] == thisone) & (events_df['visitor_or_home'] == 0)]['visteam'].drop_duplicates().tolist()
    teamlist += events_df[(events_df['batterID'] == thisone) & (events_df['visitor_or_home'] == 1)]['hometeam'].drop_duplicates().tolist()
    teamlist += events_df[(events_df['pitcherID'] == thisone) & (events_df['visitor_or_home'] == 1)]['visteam'].drop_duplicates().tolist()
    teamlist += events_df[(events_df['pitcherID'] == thisone) & (events_df['visitor_or_home'] == 0)]['hometeam'].drop_duplicates().tolist()
    teamlist = list(set(teamlist))
    
    for thisteam in teamlist:
        max_appearances = -1
        for k,v in positions_dict.items():
            if (len(
                    events_df[
                        (events_df[k] == thisone) 
                        & (events_df['visitor_or_home'] == 0) 
                        & (events_df['hometeam'] == thisteam)
                    ]
                ) + len(
                    events_df[
                        (events_df[k] == thisone) 
                        & (events_df['visitor_or_home'] == 0) 
                        & (events_df['hometeam'] == thisteam)
                    ]
                ) > max_appearances
            ):
                primary_position = v
                max_appearances = len(
                        events_df[
                            (events_df[k] == thisone) 
                            & (events_df['visitor_or_home'] == 0) 
                            & (events_df['hometeam'] == thisteam)
                        ]
                    ) + len(
                        events_df[
                            (events_df[k] == thisone) 
                            & (events_df['visitor_or_home'] == 0) 
                            & (events_df['hometeam'] == thisteam)
                        ]
                    )
        roster_fixer_text += "rosters_df = add_roster_entry_if_needed(rosters_df, '{0:}', '{1:}', '{2:}', '{3:}', '{4:}', '{5:}', '{6:}', {7:.0f})\n".format(thisone, lastname, firstname, bathand, throwhand, thisteam, primary_position, thisyear)

if (len(roster_fixer_text) > 0):
    print('\n')   
    print(roster_fixer_text)
if (len(b_events_fixer_text) > 0):
    print('\n')   
    print(b_events_fixer_text)
if (len(p_events_fixer_text) > 0):
    print('\n')   
    print(p_events_fixer_text)

e = time.time()
g += e-s
print('\n')
print('Assigned hands in {0:,.1f} seconds!'.format(e-s))


getting roster file...
getting biodata...
	adding players not in roster file...
	adding new columns for year and teams...
	assigning hands...
	assigning pitcher hands...
	fixing special case hands...
	accounting for switch hitters...
	applying adjustments to batter and pitcher hands as needed...


No batter hand found for 0 batters...
No pitcher hand found for 0 pitchers...


Searching for batters and pitchers missing from roster files and/or with missing hands...




Assigned hands in 3.7 seconds!


In [23]:
s = time.time()
print('saving data...')
if (just_this_team_flag):
    game_info_df.to_csv('game_info_{0:}{1:}_{2:}.csv'.format(thisyear, league.lower(), just_this_team.lower()), encoding='utf-8')
    er_df.to_csv('er_{0:}{1:}_{2:}.csv'.format(thisyear, league.lower(), just_this_team.lower()), encoding='utf-8')
    events_df.to_csv('events_{0:}{1:}_{2:}.csv'.format(thisyear, league.lower(), just_this_team.lower()), encoding='utf-8')
else:
    game_info_df.to_csv('game_info_{0:}{1:}.csv'.format(thisyear, league.lower()), encoding='utf-8')
    er_df.to_csv('er_{0:}{1:}.csv'.format(thisyear, league.lower()), encoding='utf-8')
    events_df.to_csv('events_{0:}{1:}.csv'.format(thisyear, league.lower()), encoding='utf-8')

# if (len(badj_at_events) > 0):
#     events_df[events_df.index.isin(badj_at_events)].to_csv('badj_events_{0:}_{1:}{2:}_{3:}.csv'.format(firstyear, finalyear, league.lower(), just_this_team.lower()), encoding='utf-8')
# else:
#     print('no badj found')
# if (len(padj_at_events) > 0):
#     events_df[events_df.index.isin(padj_at_events)].to_csv('padj_events_{0:}_{1:}{2:}_{3:}.csv'.format(firstyear, finalyear, league.lower(), just_this_team.lower()), encoding='utf-8')
# else:
#     print('no padj found')
# if (len(ladj_at_events) > 0):
#     events_df[events_df.index.isin(ladj_at_events)].to_csv('ladj_events_{0:}_{1:}{2:}_{3:}.csv'.format(firstyear, finalyear, league.lower(), just_this_team.lower()), encoding='utf-8')
# else:
#     print('no ladj found')

e = time.time()
g = g + (e-s)
print('Done in {0:,.0f} minutes {1:,.0f} seconds!'.format(np.floor(g/60), g%60))


saving data...
Done in 37 minutes 30 seconds!


In [ ]:
# AL
# ANA 0
# BAL 1
# BOS 2
# CHA 3
# CLE 4
# DET 5
# HOU 6
# KCA 7
# MIN 8
# NYA 9
# OAK 10
# SEA 11
# TBA 12
# TEX 13
# TOR 14

# NL
# ARI 0
# ATL 1
# CHN 2
# CIN 3
# COL 4
# LAN 5
# MIA 6
# MIL 7
# NYN 8
# PHI 9
# PIT 10
# SDN 11
# SFN 12
# SLN 13
# WAS 14






### Batting/pitching adjustments previously processed seasons (long)